In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import apiKeyGoogle

In [2]:
#Configure API key

gmaps.configure(api_key=apiKeyGoogle)

In [3]:
cityCsv="../WeatherPy/cityFrame.csv"
cityFrame=pd.read_csv(cityCsv)
cityFrame

,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date
0,rikitea,-23.1203,-134.9692,79.32,78,10,17.60,PF,1613317351
1,ponta do sol,32.6667,-17.1000,64.40,72,0,4.61,PT,1613317383
2,mataura,-46.1927,168.8643,57.00,93,48,5.99,NZ,1613317374
3,flinders,-34.5833,150.8552,64.99,84,32,1.99,AU,1613317454
4,spornoye,50.8448,37.2913,12.20,73,90,11.18,RU,1613317455
...,...,...,...,...,...,...,...,...,...
557,adjumani,3.3779,31.7909,90.37,23,79,1.07,UG,1613317273
558,catio,11.2833,-15.2500,88.25,37,1,12.12,GW,1613317571
559,yaan,7.3833,8.5667,101.84,19,2,2.15,NG,1613317571
560,aksehir,38.3575,31.4164,45.48,58,4,6.60,TR,1613317571


In [4]:
locations=cityFrame[['Lat', 'Lng']]

humidity=cityFrame['Humidity']

maxIntensity=cityFrame['Humidity'].max()

fig=gmaps.figure()

heatLayer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxIntensity, point_radius=3)

fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Filter dataframe for only temperatures between 70 and 80

niceCityFrame=cityFrame.loc[(cityFrame['MaxTemp']>70)&(cityFrame['MaxTemp']<80),:]

In [6]:
#Filter dataframe for only windspeed less than 10

niceCityFrame=niceCityFrame.loc[niceCityFrame['WindSpeed']<10, :]

In [7]:
#Filter dataframe for cloudiness equal to 0

niceCityFrame=niceCityFrame.loc[niceCityFrame['Cloudiness']==0, :]

In [8]:
#Write filtered dataframe to new dataframe

newCityFrame=niceCityFrame

In [9]:
#Add hotel column

newCityFrame['Hotel']=''
newCityFrame.reset_index(drop=True)


,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel
0,sale,34.0389,-6.8166,75.20,33,0,9.22,MA,1613317256,
1,pisco,-13.7000,-76.2167,73.99,78,0,9.22,PE,1613317456,
2,akyab,20.1500,92.9000,70.59,61,0,9.73,MM,1613317486,
3,carnarvon,-24.8667,113.6333,78.80,100,0,6.91,AU,1613317388,
4,umm lajj,25.0213,37.2685,77.79,57,0,4.52,SA,1613317513,
5,sabha,27.0377,14.4283,72.25,22,0,9.46,LY,1613317527,
6,salalah,17.0151,54.0924,77.00,69,0,5.75,OM,1613317387,
7,tecoanapa,16.5167,-98.7500,78.51,40,0,2.82,MX,1613317557,


In [10]:
#Construct base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Define parameters
params = { 
    "radius":"5000",
    "type": "hotel",
    "key": apiKeyGoogle,
}

#Iterate through each row in data frame

for index, row in newCityFrame.iterrows():
    
    #Get latitude and longitude from dataframe
    
    lat=newCityFrame.loc[index, 'Lat']
    lng=newCityFrame.loc[index, 'Lng']
    
    #Pass in latitude and longitude to parameters
    
    params['location']=f"{lat},{lng}" 
    
    #Get json response and hotel name
    
    response=requests.get(base_url, params=params).json()
    
    hotelName=response['results'][0]['name']
    
    #Map hotel name to data frame
    
    newCityFrame.loc[index, 'Hotel']=hotelName
    
newCityFrame

,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel
6,sale,34.0389,-6.8166,75.20,33,0,9.22,MA,1613317256,Rabat
10,pisco,-13.7000,-76.2167,73.99,78,0,9.22,PE,1613317456,Pisco
167,akyab,20.1500,92.9000,70.59,61,0,9.73,MM,1613317486,Sittwe
201,carnarvon,-24.8667,113.6333,78.80,100,0,6.91,AU,1613317388,Carnarvon
284,umm lajj,25.0213,37.2685,77.79,57,0,4.52,SA,1613317513,Umluj
352,sabha,27.0377,14.4283,72.25,22,0,9.46,LY,1613317527,Sabhā
382,salalah,17.0151,54.0924,77.00,69,0,5.75,OM,1613317387,Salalah
503,tecoanapa,16.5167,-98.7500,78.51,40,0,2.82,MX,1613317557,Barra de Tecoanapa


In [11]:
#Get coordinates using new dataframe
coordinates=newCityFrame[['Lat', 'Lng']]

#layer markers on the map based on the coordinates

markers=gmaps.marker_layer(coordinates)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotelInfo = [info_box_template.format(**row) for index, row in newCityFrame.iterrows()]
locations = newCityFrame[["Lat", "Lng"]]

markerLayer = gmaps.marker_layer(locations, info_box_content=hotelInfo)
fig.add_layer(markerLayer)
fig

Figure(layout=FigureLayout(height='420px'))